In [ ]:
import numpy as np
import pandas as pd
from pygama import lh5
import matplotlib.pyplot as plt

In [ ]:
# Load Data

group_path = '/icpcs/icpc1/dsp'
det_name = 'P43415A'

f_list = ['dsp/Run70*_dsp.lh5']
#f_list = ['dsp/Run700[567]?_dsp.lh5']
par_list = ['trapEftp', 'dcr']
df = lh5.load_dfs(f_list, par_list, group_path, verbose=False)
df

In [ ]:
#check that the E spectrum looks good
plt.hist(df['trapEftp'], bins=1500)
plt.yscale('log')
plt.show()

In [ ]:
# from the previous plot we get the following ranges for the 2615 peak zoom:
tl_range = (6310,6380)
tl_bins = 70

# Zoom in on 2615
hist = plt.hist(df['trapEftp'], bins=tl_bins, range=tl_range)
plt.show()

# Calculate the FWHM
bins = hist[1]
bin_centers = 0.5*(bins[:-1] + bins[1:])
hist = hist[0]
imode = np.argmax(hist)
mode = bin_centers[imode]
print("max at", mode)
E_scale = 2614.5/mode
ineg = imode
while(hist[ineg]>hist[imode]/2): ineg -= 1
ipos = imode
while(hist[ipos]>hist[imode]/2): ipos += 1
print(f'fwhm = {(bins[ipos-1] - bins[ineg])*E_scale:0.3g} keV')

In [ ]:
# Look at dcr for use in charge trapping correction
plt.plot(df['trapEftp'], df['dcr'], '.')
plt.xlim(*tl_range)
plt.ylim(-10,20)
plt.show()

In [ ]:
# Get a rough charge trapping correction from the above plot
ctc = 2
tl_shift = 20
tl_dcr_range = (tl_range[0]+tl_shift, tl_range[1]+tl_shift)

# Now scan to get the best value
fracs = np.linspace(0.8, 1.2, 11)
for frac in fracs:
    ctcf = frac*ctc
    plt.hist((df['trapEftp'] + ctcf*df['dcr']+(1-frac)*100), bins=tl_bins, range=tl_dcr_range, histtype='step', label=f'{ctcf:.3g}')
plt.legend()
plt.show()
df['trapEftp'] + ctcf*df['dcr']

In [ ]:
# Now build the charge-trapping-corrected energy column
# E-dependence was eyeballed looking at the plots below
ctc = 1.9
df['ctcE'] = df['trapEftp'] + ctc*df['dcr']*(df['trapEftp']+3000)/(6360+3000)
df

In [ ]:
# plot the Tl208 peak...
hist = plt.hist(df['ctcE'], bins=tl_bins*4, range=tl_dcr_range)
plt.show()

#... and measure its FWHM
bins = hist[1]
bin_centers = 0.5*(bins[:-1] + bins[1:])
hist = hist[0]
imode = np.argmax(hist)
mode = bin_centers[imode]
print("max at", mode)
E_scale = 2614.5/mode
ineg = imode
while(hist[ineg]>hist[imode]/2): ineg -= 1
ipos = imode
while(hist[ipos]>hist[imode]/2): ipos += 1
print(f'fwhm = {(bins[ipos-1] - bins[ineg])*E_scale:0.3g} keV')

In [ ]:
# Check "by eye" how the ctc works across all energies.

plt.figure().set_dpi(200)
ms = 0.05
fs = 'small'
tfs = 'xx-small'

plt.subplot(331)
plt.text(238, 4, '$^{212}$Pb', size=fs)
plt.plot(df['trapEftp']*E_scale, df['dcr'], '.', markersize=ms/2)
plt.plot(df['ctcE']*E_scale, df['dcr'], '.', markersize=ms/2)
plt.xlim(220,250) 
plt.gca().set_ylabel('DCR')
plt.ylim(-5,10)
plt.gca().tick_params(axis='both', which='major', labelsize=tfs)

plt.subplot(332)
plt.text(510, 3, '$^{208}$Tl', size=fs)
plt.text(583, 5, '$^{208}$Tl', size=fs, ha='center')
plt.plot(df['trapEftp']*E_scale, df['dcr'], '.', markersize=ms/2)
plt.plot(df['ctcE']*E_scale, df['dcr'], '.', markersize=ms/2)
plt.xlim(500,600)
plt.ylim(-5,10)
plt.gca().tick_params(axis='both', which='major', labelsize=tfs)

plt.subplot(333)
plt.text(670, 5, 'pulser', size=fs)
plt.plot(df['trapEftp']*E_scale, df['dcr'], '.', markersize=ms)
plt.plot(df['ctcE']*E_scale, df['dcr'], '.', markersize=ms)
plt.xlim(660,680)
plt.ylim(-5,10)
plt.gca().tick_params(axis='both', which='major', labelsize=tfs)

plt.subplot(334)
plt.text(727, 5, '$^{212}$Bi')
plt.plot(df['trapEftp']*E_scale, df['dcr'], '.', markersize=ms)
plt.plot(df['ctcE']*E_scale, df['dcr'], '.', markersize=ms)
plt.xlim(700,750)
plt.gca().set_ylabel('DCR')
plt.ylim(-5,10)
plt.gca().tick_params(axis='both', which='major', labelsize=tfs)

plt.subplot(335)
plt.text(860, 5, '$^{208}$Tl')
plt.plot(df['trapEftp']*E_scale, df['dcr'], '.', markersize=ms)
plt.plot(df['ctcE']*E_scale, df['dcr'], '.', markersize=ms)
plt.xlim(840,900)
plt.ylim(-5,10)
plt.gca().tick_params(axis='both', which='major', labelsize=tfs)

plt.subplot(337)
plt.text(1592, 10, 'DEP', ha='right')
plt.text(1620, 10, '$^{212}$Bi', ha='center')
plt.plot(df['trapEftp']*E_scale, df['dcr'], '.', markersize=ms*2)
plt.plot(df['ctcE']*E_scale, df['dcr'], '.', markersize=ms*2)
plt.gca().set_xlabel('E [keV]')
plt.xlim(1560,1640)
plt.gca().set_ylabel('DCR')
plt.ylim(-10,20)
plt.gca().tick_params(axis='both', which='major', labelsize=tfs)

plt.subplot(338)
plt.text(2104, 15, 'SEP')
plt.plot(df['trapEftp']*E_scale, df['dcr'], '.', markersize=ms)
plt.plot(df['ctcE']*E_scale, df['dcr'], '.', markersize=ms)
plt.gca().set_xlabel('E [keV]')
plt.xlim(2000,2200)
plt.ylim(-10,20)
plt.gca().tick_params(axis='both', which='major', labelsize=tfs)

plt.subplot(339)
plt.text(2615, 15, '$^{208}$Tl')
plt.plot(df['trapEftp']*E_scale, df['dcr'], '.', markersize=ms)
plt.plot(df['ctcE']*E_scale, df['dcr'], '.', markersize=ms)
plt.gca().set_xlabel('E [keV]')
plt.xlim(2600,2630) # 208Tl 2615
plt.ylim(-10,20)
plt.gca().tick_params(axis='both', which='major', labelsize=tfs)

plt.show()